# Used-Cars

## Programming for Data Science Course from University of Science
## Final project



### Team members
<center>

|Thành viên           |MSSV     |
|:-------------------|:-------|
|Vũ Minh Chiến        |22127045 |
|Quách Trần Quán Vinh |22127460 |

</center>

### **1. Collecting data**

What subject is your data about?
- Craigslist is the world's largest collection of used vehicles for sale, this dataset includes part of used vehicle entry within the United States on Craigslist.

What is the source of your data?
- The source of this data set can be found in [kaggle](https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data/data). For this project, we have reduce the origin data set, which contains 426880 rows, down to 150000 rows (the reason for this will be mentioned in reflection part).

Do authors of this data allow you to use like this?
- Yes, this data set have `CC0: Public Domain` license (you can find more informaitons about the license [here](https://creativecommons.org/publicdomain/zero/1.0/)).

How did authors collect data?
- The arthor collect data by a web application built with Flask which scrapes used vehicle listings weekly from all regional Craigslist sites in the United States (you can find more informaitons on scraping process [here](https://github.com/AustinReese/UsedVehicleSearch)).

Import necessary libraries

In [7]:
import zipfile
import pandas as pd

Read the dataset

In [8]:
df = pd.read_csv('vehicles.zip', delimiter=',',compression='zip')

### **2. Exploring data**

In [9]:
# display dataset
df.sample(5)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
97955,7313751282,https://winstonsalem.craigslist.org/ctd/d/wins...,winston-salem,https://winstonsalem.craigslist.org,23590,2017.0,nissan,frontier king cab sv,good,NaN,...,NaN,pickup,red,https://images.craigslist.org/00R0R_aLtwcjHyVj...,Carvana is the safer way to buy a car During t...,NaN,nc,36.100000,-80.240000,2021-04-28T08:21:16-0400
23646,7312385581,https://desmoines.craigslist.org/ctd/d/des-moi...,des moines,https://desmoines.craigslist.org,0,1949.0,chrysler,windsor,excellent,NaN,...,NaN,NaN,green,https://images.craigslist.org/00R0R_bO2wthKNFD...,1949 *** CHRYSLER WINDSOR *** Drive it hom...,NaN,ia,41.542262,-93.625184,2021-04-25T12:00:23-0500
50688,7310398472,https://maine.craigslist.org/ctd/d/burnham-201...,maine,https://maine.craigslist.org,14955,2010.0,jeep,wrangler,NaN,NaN,...,NaN,NaN,green,https://images.craigslist.org/00R0R_fXStqWpoUf...,Second Street Auto - 1000 Second Street - Manc...,NaN,me,44.693199,-69.334602,2021-04-21T12:53:58-0400
13772,7305822732,https://charlestonwv.craigslist.org/ctd/d/chil...,charleston,https://charlestonwv.craigslist.org,26988,2015.0,ford,f-350sd,excellent,NaN,...,NaN,pickup,NaN,https://images.craigslist.org/01414_gv6NYSJOmm...,Drive it home today. Call us now !!Call ☏ (740...,NaN,wv,39.334610,-82.949321,2021-04-12T16:05:05-0400
90208,7314348429,https://topeka.craigslist.org/ctd/d/topeka-200...,topeka,https://topeka.craigslist.org,4850,2005.0,ford,ranger edge,good,6 cylinders,...,mid-size,pickup,grey,https://images.craigslist.org/00l0l_437Nt2DLRW...,2005 Ford Ranger 3.0 5 Speed Short Bed very ni...,NaN,ks,39.058300,-95.709500,2021-04-29T11:03:15-0500


##### How many rows and columns?

In [10]:
df.shape

(100000, 26)

##### What is the meaning of each row/column?

- Please refer to [kaggle](https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data/data) in order to get the meaning of each column.
- Each row corresponds to an used car with different attributes being sale.

##### Are there any duplicated rows?

In [11]:
df.duplicated().sum()

np.int64(0)

##### What is the current data type of each column? Are there columns having inappropriate data types?

In [13]:
df.dtypes

id                int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
type             object
paint_color      object
image_url        object
description      object
county          float64
state            object
lat             float64
long            float64
posting_date     object
dtype: object

`county` should be object type instead of float64 type.

##### With each numerical column, how are values distributed?

##### With each categorical column, how are values distributed?

### **3. Asking meaningful questions**

##### What factors most significantly impact used car prices?
##### How does vehicle price vary with odometer reading across manufacturers?
##### What is the relationship between vehicle condition and posting date (e.g., do sellers list damaged vehicles more often at certain times of the year)?
##### What is the average time between a vehicle's manufacturing year and its sale listing date? How does this vary by type, region, or price?
##### What are the most common paint colors for specific vehicle types, and how does this affect their resale value?
##### Which manufacturers or models tend to dominate specific regions, and how does this affect their pricing?
##### How does the presence of a VIN (Vehicle Identification Number) affect vehicle pricing and perceived quality?